# Выгрузка данных с локального хранилища на яндекс диск

Для работы необходимо:
* зарегестрировать приложение на яндекс OAuth (по ссылке [https://oauth.yandex.ru](https://oauth.yandex.ru)) с правами на чтение и запись в яндекс диск через REST API
* запомнить `ClientID`
* получить OAuth-токен для внешних приложений по ссылке [https://oauth.yandex.ru/authorize?response_type=token&client_id={ClientID}](https://oauth.yandex.ru/authorize?response_type=token&client_id={ClientID})
* записать этот токен в глобальную переменную `TOKEN`

In [ ]:
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
import yadisk

TOKEN = '<your_yadisk_token>'

In [ ]:
client = yadisk.YaDisk(token=TOKEN)

with client:
    print(client.check_token())

#### Cоздание папки

In [ ]:
with client:
    client.mkdir('/desert_data/yandexmaps_data/sample_dir')

#### Выгрузка файлов из директории: однопоточный вариант

In [ ]:
N = 200000
errors = []

with client:
    for ind in tqdm(range(N)):
        try:
            local_file_name = f'../yandexmaps_data/negative_sampels/img{str(ind).zfill(6)}.png'
            with open(local_file_name, 'rb') as f:
                client.upload(f, f'/desert_data/yandexmaps_data/negative_samples_200k/img{str(ind).zfill(6)}.png')ч
        except Exception as e:
            errors.append(ind)
            continue

print(f'{len(errors)=}')

#### Выгрузка файлов из директории: многопоточный вариант

In [ ]:
N = 200000
errors = []

def upload_file(client, ind):
    try:
        local_file_name = f'/Users/nadys/PythonProjects/archaeology-project/data_retrieval/yandexmaps_data/negative_sampels/img{str(ind).zfill(6)}.png'
        with open(local_file_name, 'rb') as f:
            client.upload(f, f'/desert_data/yandexmaps_data/negative_samples_200k/img{str(ind).zfill(6)}.png')
    except Exception as e:
        errors.append(ind)

with client, ThreadPoolExecutor() as executor:
    futures = {executor.submit(upload_file, client, ind) for ind in range(N)}
    for i, future in enumerate(futures):
        pass

print(f'{len(errors)=}')